In [16]:
import torch
import transformers
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import wikipedia
import nltk

#utile pour le pos tagging
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [18]:
# == not useful, Only for print token and id ==

# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.

def print_token_id(input_ids):
    # For each token and its id...
    for token, id in zip(tokens, input_ids):
        # If this is the [SEP] token, add some space around it to make it stand out.
        if id == tokenizer.sep_token_id:
            print('')
        # Print the token string and its ID in two columns.
        print('{:<12} {:>6,}'.format(token, id))
        if id == tokenizer.sep_token_id:
            print('')

#tokens = tokenizer.convert_ids_to_tokens(input_ids)
            

In [19]:
def generate_answer(question, answer_text):
    # == Tokenize ==
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text)
    print('The input has a total of {:} tokens.'.format(len(input_ids)))

    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    # == Set Segment IDs ==
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # == Run Model ==
    # Run our example through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                 token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from 

    
    # donc on applique un argmax pour trouver le plus probable
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # == Print Answer without ## ==
    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
    
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
    
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')

    

In [20]:
# ============================================================================================================================ #

In [24]:
# Fonction qui extrait le dernier sujet de la question
def extract_subject(question):
    subject = None
    token = nltk.word_tokenize(question)
    print(token)
    pos_token = nltk.pos_tag(token)
    for item in pos_token:
        print(item)
        if item[1] == 'NN' or item[1] == 'NNP':
            subject = item[0]
    return  subject

# == WIP ==
# découpé en paragraphe (le model a une limite de 512 token pour le text en entré)
def get_wiki_and_split(subject):
    answer_text = wikipedia.summary(subject)
    return data

In [27]:
#Enter HERE the question you want :
question = "Who is Barack Obama ?"

subject = extract_subject(question)

print("The Subject is : "+subject)
print("")

if subject is not None:
    answer_text = wikipedia.summary(subject)
    generate_answer(question, answer_text[:2000])
    print(answer_text[:500])
else:
    print("Subject not found :/")

['Who', 'is', 'Barack', 'Obama', '?']
('Who', 'WP')
('is', 'VBZ')
('Barack', 'NNP')
('Obama', 'NNP')
('?', '.')
The Subject is : Obama

The input has a total of 417 tokens.
Answer: "barack hussein obama ii"
Barack Hussein Obama II ( (listen) bə-RAHK hoo-SAYN oh-BAH-mə; born August 4, 1961) is an American politician and attorney who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic Party, Obama was the first African-American president of the United States. He previously served as a U.S. senator from Illinois from 2005 to 2008 and an Illinois state senator from 1997 to 2004.
Obama was born in Honolulu, Hawaii. After graduating from Columbia University in 
